# This notebook provides portfolio weights based on the historical data to the backtesting notebook

In [ ]:
!pip uninstall deepdowmine -y
!pip install git+https://github.com/dsman1823/deepdowmine.git

In [1]:
import torch

import pandas as pd
import numpy as np

from datetime import datetime 
from deepdowmine.nn import LinearNetMine, UpdNumericalMarkowitzWithShorting, UpdLinearNetMine, UpdDenseNet, UpdUpdLinearNetMine
from deepdowmine.nn import DenseNetMinVar, ConvNetFullOpti, ConvNetMinVar, RnnNetMinVar, LstmNetMinVar, RnnNetFullOpti
from deepdowmine.nn import LstmNetFullOpti, DenseNetFullOpti2, DenseNetMinVar2, RnnNetMinVar2, RnnNetFullOpti2, ConvNetMinVar2
from dateutil.relativedelta import relativedelta
from datetime import datetime, date

In [2]:
RETS_FILE_PATH = 'historical_returns.csv'



loockback, gap, horizon = 50, 0, 5
n_assets = 5
loockback = 50


## Load NN from dict

In [3]:
LINEAR_NET_OSHARPE_FILE_PATH = r'./NNs/linear_net_50x5_sharpe.pth' # LinearNetMine with ordinary Sharpe cost

In [4]:
# years parameter reprsenets amount of years after the backtest stasrt: the model dowloaded is the the model trained 
# on the period (train_start, back_test_start + years) 


# LinearNetMine with ordinary sharpe cost function
def load_linear_net_osharpe(years=0):
    network = LinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{years}.pth'))
    print(fr'./NNs/linear_net_50x5_sharpe_{years}.pth')
    return network.eval()
    

In [5]:
def load_dense_net(years=0):
    network = UpdLinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/dense_{years}.pth'))
    print(fr'dense_{years}.pth')
    return network.eval()

## Core functions

In [6]:
def transform_rets_to_NN_input(rets):
    # tranform (loockback, n_assets) df into (1, 1, loockback, n_assets)=(sample_size, n_channels, loockback, n_assets) tensor 
    returns_np = rets.to_numpy()

    # Add the required dimensions: (n_samples, n_channels, lookback, n_assets)
    returns_np_expanded = np.expand_dims(returns_np, axis=0)  # Adds n_samples dimension
    returns_np_expanded = np.expand_dims(returns_np_expanded, axis=0)  
    return torch.from_numpy(returns_np_expanded).float()

def weights_from_NN(rets, network):
    X = transform_rets_to_NN_input(rets)
    return network(X).detach().numpy()[0]

# method is a function, used for obtaining the returns
def get_weights(method):
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    return method(rets)

## NNs and related methods


### NNs

networks[i] method is trained on (train_start, backtest_start + i years) data interval

In [7]:
years = [0, 1, 2]#, 1, 2]

linear_osharpe_networks = [load_linear_net_osharpe(years=i) for i in years]
dense_networks = [load_dense_net(years=i) for i in years]

./NNs/linear_net_50x5_sharpe_0.pth
./NNs/linear_net_50x5_sharpe_1.pth
./NNs/linear_net_50x5_sharpe_2.pth
dense_0.pth
dense_1.pth
dense_2.pth


### methods

In [8]:
dense_networks[1].linear0.bias[0:3]

tensor([ 0.0484,  0.0029, -0.0442], grad_fn=<SliceBackward0>)

In [9]:
dense_networks[2].linear0.bias[0:3]

tensor([ 0.0484,  0.0029, -0.0442], grad_fn=<SliceBackward0>)

In [10]:
# note the format of lambda 
linear_osharpe_methods = [lambda r, i=i: weights_from_NN(r, linear_osharpe_networks[i]) for i in years] 
dense_methods = [lambda r, i=i: weights_from_NN(r, dense_networks[i]) for i in years] 


In [11]:
[get_weights(dense_methods[i]) for i in years]

[array([0.01819081, 0.24908274, 0.29753768, 0.41258937, 0.0225994 ],
       dtype=float32),
 array([-0.03791929,  0.1064674 ,  0.4017897 ,  0.46670547,  0.06295667],
       dtype=float32),
 array([-0.03175861,  0.18791872,  0.14897145,  0.48070765,  0.21416074],
       dtype=float32)]

# Server

In [12]:


# RETRAINING_DATE2018 = date(2018, 12, 15)
# RETRAINING_DATE2019 = date(2019, 12, 15)

RETRAINING_DATE1 = date(2021, 12, 15)
RETRAINING_DATE2 = date(2022, 12, 15)

In [13]:
from flask import Flask, request, jsonify
from flask_cors import CORS 

import numpy as np

app = Flask(__name__)
CORS(app)  # Enable CORS for your Flask app

markers = {}
markers['net_initialized'] = False

@app.route('/get_weights', methods=['GET'])
def get_request():
    # Extract date string from the query parameters
    date_str = request.args.get('date', None)
    
    # Simple validation to check if date is provided
    if not date_str:
        return jsonify({"error": "Missing date parameter"}), 400

    # Try to convert the date string to a datetime object
    try:
        # Note that now we're only parsing the date, not the time
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        # If there is an error in parsing the date, return an error message
        return jsonify({"error": "Invalid date format. Please use YYYY-MM-DD format."}), 400
    
    years_plus = 0
    network = ConvNetMinVar2(1, 50, 5, .15, 1)
    network.load_state_dict(torch.load(fr'./NNs/trained_on_new_data/conv/min_var_0.pth'))

    
    if date < RETRAINING_DATE1:
        print(0)
    if RETRAINING_DATE1 < date < RETRAINING_DATE2:
        print('1')
        network = ConvNetMinVar2(1, 50, 5, .15, 1)
        network.load_state_dict(torch.load(fr'./NNs/trained_on_new_data/conv/min_var_1.pth'))
    if RETRAINING_DATE2 < date:
        print('2')
        network = ConvNetMinVar2(1, 50, 5, .15, 1)
        network.load_state_dict(torch.load(fr'./NNs/trained_on_new_data/conv/min_var_2.pth'))

    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    X = transform_rets_to_NN_input(rets)
    weights = network.eval()(X).detach().numpy()[0].tolist()
    
        
    #weights = weights_from_NN(rets, linear_osharpe_networks[0]).tolist()
    #print(weights)
    # Return the vector as JSON, using the string representation of the date for simplicity
    print(date)
    print(weights)
    return jsonify({'weights': weights})


In [ ]:
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Apr/2024 00:48:00] "GET /get_weights?date=2020-12-21 HTTP/1.1" 200 -


0
2020-12-21
[-0.3147144913673401, 0.9999938607215881, -0.19605451822280884, 0.48103758692741394, 0.029737472534179688]


127.0.0.1 - - [07/Apr/2024 00:48:02] "GET /get_weights?date=2020-12-28 HTTP/1.1" 200 -


0
2020-12-28
[0.06423242390155792, 0.201311394572258, -0.22191432118415833, 0.9754660129547119, -0.019095532596111298]


...

2
2023-11-27
[-0.086556576192379, 0.6386826634407043, -0.08685964345932007, 0.03270023688673973, 0.5020332932472229]


127.0.0.1 - - [07/Apr/2024 00:59:22] "GET /get_weights?date=2023-12-04 HTTP/1.1" 200 -


2
2023-12-04
[-0.45203453302383423, 0.8757162094116211, 0.07389269024133682, -0.02911693975329399, 0.5315425395965576]


127.0.0.1 - - [07/Apr/2024 00:59:24] "GET /get_weights?date=2023-12-11 HTTP/1.1" 200 -


2
2023-12-11
[0.11899766325950623, 0.2221270352602005, 0.19582408666610718, 0.6708230376243591, -0.20777182281017303]


In [ ]:
update the 